In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals


import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds


print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "Not Availiable")




Version:  2.2.0
Eager mode:  True
Hub version:  0.8.0
GPU is Not Availiable


In [2]:
# Split the training set into 60% and 40%, 
# end up 15,000 examples for training, 10,000 examples for validataion and 25,000
# examples for testing:




train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5XDWSG/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5XDWSG/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5XDWSG/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


# Explore the data

Usually, print out some random / initial stuff to see what's out data look like

In [0]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [4]:
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [5]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

#Build the model

architectural decisions to be made
1. How to represent the text?
2. How many layers to use in the model?
3. How many hidden units to use for each layer?



Input data-> sentences
labels-> 0 or 1

To represent the text, we need to convert the sentences into embedding vectors. We can use a pre-trained text embedding as first layer, it gives us 3 advantages
1. don't have to worry about preprocessing
2. benefit from transfer learning
3. the embedding has a fixed size, so it's simpler to process



In [6]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype = tf.string, trainable=True)

hub_layer(train_examples_batch[:3])



<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

the **layers**


1. Tensorflow Hub layer. 
The layer uses a pre-trained saved model to map a sentence into its embedding vector. The pre-trained text embedding model splits the sentence to tokens, embeds each token and then combines the embedding. No matter what the input is, the resulting dimensions are (num_examples, embedding_dimension)



2. The fixed-length output vector is piped through a fully-connected(Dense) layer with 16 hidden units.

3. The last layer is densely coneected with a single output node

In [7]:
# build the full model

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


**Compile the model**

A model needs a loss function and an optimizer for training. 

Since this is a binary classification problem and the model outputs a probability(single-unit layer with a sigmoid activation), we'll use the **binary-crossentropy** loss function

Generally, *binary_crossentropy* is better dealing with probabilities-- it measures the "distance" between probability distributions, or in our case, between the ground-truth distribution and the predictions. 



In [0]:
model.compile(optimizer='adam', 
              loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=['accuracy'])

#Train the model
Epochs: 20
batch samples: 512

This is 20 iterations over all samples in the x_train and y_train tensors. 

While training, monitor the model's loss and accuracy on the 10,000 from the validation set



In [9]:
history = model.fit(train_data.shuffle(10000).batch(512), 
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 3s 103ms/step - loss: 0.9828 - accuracy: 0.5120 - val_loss: 0.7277 - val_accuracy: 0.5440
Epoch 2/20
30/30 [==============================] - 3s 98ms/step - loss: 0.7073 - accuracy: 0.5635 - val_loss: 0.6830 - val_accuracy: 0.5759
Epoch 3/20
30/30 [==============================] - 3s 99ms/step - loss: 0.6630 - accuracy: 0.5863 - val_loss: 0.6485 - val_accuracy: 0.6079
Epoch 4/20
30/30 [==============================] - 3s 97ms/step - loss: 0.6296 - accuracy: 0.6222 - val_loss: 0.6161 - val_accuracy: 0.6362
Epoch 5/20
30/30 [==============================] - 3s 97ms/step - loss: 0.5936 - accuracy: 0.6583 - val_loss: 0.5811 - val_accuracy: 0.6689
Epoch 6/20
30/30 [==============================] - 3s 99ms/step - loss: 0.5625 - accuracy: 0.6822 - val_loss: 0.5530 - val_accuracy: 0.6925
Epoch 7/20
30/30 [==============================] - 3s 100ms/step - loss: 0.5322 - accuracy: 0.7080 - val_loss: 0.5246 - val_accuracy: 0.7220
Epoch 8/20


# Evaluate the model

see the returned Loss and Accuracy use the test batch

In [10]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 3s - loss: 0.3151 - accuracy: 0.8569
loss: 0.315
accuracy: 0.857


Current accuracy is around 85, with more precise preprocess and advanced approach, it should approach 95% accuracy